
Using dbutils.fs.ls command to check the file that been uploaded

In [0]:
#Using dbutils.fs.ls command to check the file that been uploaded

dbutils.fs.ls("/FileStore/tables/")

Out[1]: [FileInfo(path='dbfs:/FileStore/tables/account-models/', name='account-models/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/accounts/', name='accounts/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/accounts.zip', name='accounts.zip', size=5297592, modificationTime=1706721501000),
 FileInfo(path='dbfs:/FileStore/tables/accounts2/', name='accounts2/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations/', name='activations/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations.zip', name='activations.zip', size=8411369, modificationTime=1706718610000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2023/', name='clinicaltrial_2023/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2023-1.zip', name='clinicaltrial_2023-1.zip', size=52785750, modificationTime=1712308832000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2023.zip', n


Defining Reusable code to unzip all files

In [0]:
#Copy unzipped file
fileroot = "clinicaltrial_2023"

dbutils.fs.cp("FileStore/tables/" + fileroot + ".zip", "file:/tmp/")

Out[2]: True

In [0]:
#import os library

import os
os.environ['fileroot'] = fileroot


In [0]:
%sh

unzip -d /tmp /tmp/$fileroot.zip

Archive:  /tmp/clinicaltrial_2023.zip
  inflating: /tmp/clinicaltrial_2023.csv  


In [0]:
# List the contents of the /tmp/ directory
files_in_tmp = dbutils.fs.ls("file:/tmp/")

# Print the list of files and directories in /tmp/
for file_info in files_in_tmp:
    print(file_info.path)

file:/tmp/hsperfdata_root/
file:/tmp/python_lsp_logs/
file:/tmp/.Test-unix/
file:/tmp/clinicaltrial_2023.zip
file:/tmp/systemd-private-79d404651054472b8560fab44e9246a0-systemd-logind.service-jtWTBg/
file:/tmp/driver-env.sh
file:/tmp/.font-unix/
file:/tmp/driver-daemon.pid
file:/tmp/Rserv/
file:/tmp/chauffeur-daemon.pid
file:/tmp/.XIM-unix/
file:/tmp/Rtmp8MREq4/
file:/tmp/tmp.LsYI7WzaBq
file:/tmp/systemd-private-79d404651054472b8560fab44e9246a0-systemd-resolved.service-cNc66e/
file:/tmp/systemd-private-79d404651054472b8560fab44e9246a0-apache2.service-AMVHhi/
file:/tmp/clinicaltrial_2023.csv
file:/tmp/systemd-private-79d404651054472b8560fab44e9246a0-ntp.service-4wlcjg/
file:/tmp/.ICE-unix/
file:/tmp/chauffeur-env.sh
file:/tmp/chauffeur-daemon-params
file:/tmp/.X11-unix/
file:/tmp/custom-spark.conf
file:/tmp/driver-daemon-params


In [0]:
#Make directory

dbutils.fs.mkdirs("FileStore/tables/" + fileroot)

Out[6]: True

In [0]:
#Move files

dbutils.fs.mv("file:/tmp/" + fileroot + ".csv" , "/FileStore/tables/" + fileroot , True)

Out[7]: True

In [0]:
# remove the .zip file

dbutils.fs.rm("/FileStore/tables/" + fileroot + ".zip")

dbutils.fs.ls("/FileStore/tables/" + fileroot)

Out[8]: [FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2023/clinicaltrial_2023.csv', name='clinicaltrial_2023.csv', size=202413369, modificationTime=1712855077000)]

In [0]:
file = "/FileStore/tables/clinicaltrial_2023/clinicaltrial_2023.csv"

dbutils.fs.head(file)

[Truncated to first 65536 bytes]
Out[9]: 'NCT Number \\t Study Title \\t Acronym \\t Study Status \\t Conditions \\t Interventions \\t Sponsor \\t Collaborators \\t Enrollment \\t Funder Type \\t Study Type \\t Study Design,Start Date,Completion Date\r\n"NCT03630471 \\t Effectiveness of a Problem-solving Intervention for Common Adolescent Mental Health Problems in India \\t PRIDE \\t COMPLETED \\t Mental Health Issue (E.G., Depression, Psychosis, Personality Disorder, Substance Abuse) \\t BEHAVIORAL: PRIDE \'Step 1\' problem-solving intervention|BEHAVIORAL: Enhanced usual care \\t Sangath \\t Harvard Medical School (HMS and HSDM)|London School of Hygiene and Tropical Medicine \\t 250 \\t OTHER \\t INTERVENTIONAL \\t Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (INVESTIGATOR, OUTCOMES_ASSESSOR)|Primary Purpose: TREATMENT",20 August 2018,28 February 2019\r\n"NCT05992571 \\t Oral Ketone Monoester Supplementation and Resting-state Brain Connectivity \\t RECRUITING \\

In [0]:
# Create rdd

rdd1 = sc.textFile(file)

rdd1.take(3)

Out[80]: ['NCT Number \\t Study Title \\t Acronym \\t Study Status \\t Conditions \\t Interventions \\t Sponsor \\t Collaborators \\t Enrollment \\t Funder Type \\t Study Type \\t Study Design,Start Date,Completion Date',
 '"NCT03630471 \\t Effectiveness of a Problem-solving Intervention for Common Adolescent Mental Health Problems in India \\t PRIDE \\t COMPLETED \\t Mental Health Issue (E.G., Depression, Psychosis, Personality Disorder, Substance Abuse) \\t BEHAVIORAL: PRIDE \'Step 1\' problem-solving intervention|BEHAVIORAL: Enhanced usual care \\t Sangath \\t Harvard Medical School (HMS and HSDM)|London School of Hygiene and Tropical Medicine \\t 250 \\t OTHER \\t INTERVENTIONAL \\t Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (INVESTIGATOR, OUTCOMES_ASSESSOR)|Primary Purpose: TREATMENT",20 August 2018,28 February 2019',
 '"NCT05992571 \\t Oral Ketone Monoester Supplementation and Resting-state Brain Connectivity \\t RECRUITING \\t Cerebrovascular Function|Co

In [0]:
#Extract header

header = rdd1.first()

print(header)

NCT Number \t Study Title \t Acronym \t Study Status \t Conditions \t Interventions \t Sponsor \t Collaborators \t Enrollment \t Funder Type \t Study Type \t Study Design,Start Date,Completion Date


In [0]:
def clean_line(line, header):
    # Split the line by tab character
    parts = line.split("\\t")
    
    # Ensure that the line has the same number of fields as the header
    num_fields = len(header)
    num_parts = len(parts)
    
    if num_parts < num_fields:
        # Append empty strings to fill missing fields
        parts += [''] * (num_fields - num_parts)
    elif num_parts > num_fields:
        # Truncate the line to match the number of fields in the header
        parts = parts[:num_fields]
    
    return parts

# Define the header
header = ['NCT Number', 'Study Title', 'Acronym', 'Study Status', 'Conditions', 
          'Interventions', 'Sponsor', 'Collaborators', 'Enrollment', 'Funder Type', 
          'Study Type', 'Study Design', 'Start Date', 'Completion Date']

# Apply the clean_line function to the input list and save as rdd2
rdd2 = rdd1.map(lambda line: clean_line(line, header))

# Verify the cleaned RDD
rdd2.take(5)

Out[69]: [['NCT Number ',
  ' Study Title ',
  ' Acronym ',
  ' Study Status ',
  ' Conditions ',
  ' Interventions ',
  ' Sponsor ',
  ' Collaborators ',
  ' Enrollment ',
  ' Funder Type ',
  ' Study Type ',
  ' Study Design,Start Date,Completion Date',
  '',
  ''],
 ['"NCT03630471 ',
  ' Effectiveness of a Problem-solving Intervention for Common Adolescent Mental Health Problems in India ',
  ' PRIDE ',
  ' COMPLETED ',
  ' Mental Health Issue (E.G., Depression, Psychosis, Personality Disorder, Substance Abuse) ',
  " BEHAVIORAL: PRIDE 'Step 1' problem-solving intervention|BEHAVIORAL: Enhanced usual care ",
  ' Sangath ',
  ' Harvard Medical School (HMS and HSDM)|London School of Hygiene and Tropical Medicine ',
  ' 250 ',
  ' OTHER ',
  ' INTERVENTIONAL ',
  ' Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (INVESTIGATOR, OUTCOMES_ASSESSOR)|Primary Purpose: TREATMENT",20 August 2018,28 February 2019',
  '',
  ''],
 ['"NCT05992571 ',
  ' Oral Ketone Monoester Sup


Data Cleaning

In [0]:
#Let's split Header

# Extract header
header = rdd1.first()

# Split the header based on \t
header_elements = header.split("\t")

# Split the last element based on comma
last_element_split = header_elements[-1].split(',')

# Combine the split elements into the header list
header_elements[-1:] = last_element_split

# Print the modified header
print(header_elements)

['NCT Number \\t Study Title \\t Acronym \\t Study Status \\t Conditions \\t Interventions \\t Sponsor \\t Collaborators \\t Enrollment \\t Funder Type \\t Study Type \\t Study Design', 'Start Date', 'Completion Date']


In [0]:
def split_line(line):
    # Split the line by tab character
    parts = line.split("\\t")
    
    # Ensure at least three parts exist
    if len(parts) < 3:
        return parts
    
    # Split the last part by commas
    last_part_split = parts[-1].split(",")
    
    # Combine the parts
    return parts[:-1] + [last_part_split[0]] + [",".join(last_part_split[1:])]

# Apply the split_line function to the input list and save as rdd2
rdd2 = rdd1.map(split_line)

rdd2.take(5)

Out[57]: [['NCT Number ',
  ' Study Title ',
  ' Acronym ',
  ' Study Status ',
  ' Conditions ',
  ' Interventions ',
  ' Sponsor ',
  ' Collaborators ',
  ' Enrollment ',
  ' Funder Type ',
  ' Study Type ',
  ' Study Design',
  'Start Date,Completion Date'],
 ['"NCT03630471 ',
  ' Effectiveness of a Problem-solving Intervention for Common Adolescent Mental Health Problems in India ',
  ' PRIDE ',
  ' COMPLETED ',
  ' Mental Health Issue (E.G., Depression, Psychosis, Personality Disorder, Substance Abuse) ',
  " BEHAVIORAL: PRIDE 'Step 1' problem-solving intervention|BEHAVIORAL: Enhanced usual care ",
  ' Sangath ',
  ' Harvard Medical School (HMS and HSDM)|London School of Hygiene and Tropical Medicine ',
  ' 250 ',
  ' OTHER ',
  ' INTERVENTIONAL ',
  ' Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (INVESTIGATOR',
  ' OUTCOMES_ASSESSOR)|Primary Purpose: TREATMENT",20 August 2018,28 February 2019'],
 ['"NCT05992571 ',
  ' Oral Ketone Monoester Supplementation a

In [0]:
#Define function to split lines in text

def split_line(line):
    # Split the line by tab character
    parts = line.split("\\t")
    # Split the last part by commas and extend the list
    last_part_split = parts[-1].split(",")
    return parts[:-1] + last_part_split

# Apply the split_line function to the input list and save as rdd2
rdd2 = rdd1.map(split_line)

rdd2.take(2)

Out[100]: [['NCT Number ',
  ' Study Title ',
  ' Acronym ',
  ' Study Status ',
  ' Conditions ',
  ' Interventions ',
  ' Sponsor ',
  ' Collaborators ',
  ' Enrollment ',
  ' Funder Type ',
  ' Study Type ',
  ' Study Design',
  'Start Date',
  'Completion Date'],
 ['"NCT03630471 ',
  ' Effectiveness of a Problem-solving Intervention for Common Adolescent Mental Health Problems in India ',
  ' PRIDE ',
  ' COMPLETED ',
  ' Mental Health Issue (E.G., Depression, Psychosis, Personality Disorder, Substance Abuse) ',
  " BEHAVIORAL: PRIDE 'Step 1' problem-solving intervention|BEHAVIORAL: Enhanced usual care ",
  ' Sangath ',
  ' Harvard Medical School (HMS and HSDM)|London School of Hygiene and Tropical Medicine ',
  ' 250 ',
  ' OTHER ',
  ' INTERVENTIONAL ',
  ' Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (INVESTIGATOR',
  ' OUTCOMES_ASSESSOR)|Primary Purpose: TREATMENT"',
  '20 August 2018',
  '28 February 2019']]

In [0]:
#Cleaning date
#Extract first line data
first_line = rdd2.map(lambda x: x[-1])

first_line.take(4)

Out[101]: ['Completion Date', '28 February 2019', '2024-08', '2014-12']

In [0]:
#Check the first row

Header_row = rdd2.take(1)

print(Header_row)

[['NCT Number ', ' Study Title ', ' Acronym ', ' Study Status ', ' Conditions ', ' Interventions ', ' Sponsor ', ' Collaborators ', ' Enrollment ', ' Funder Type ', ' Study Type ', ' Study Design', 'Start Date', 'Completion Date']]


In [0]:
# Extract Start Date and Completion Date

rdd3 = rdd2.map(lambda row: (row[-2], row[-1]))

rdd3.collect()  #show all data in rdd to examine date format that exist

Out[103]: [('Start Date', 'Completion Date'),
 ('20 August 2018', '28 February 2019'),
 ('25 October 2023', '2024-08'),
 ('2013-08', '2014-12'),
 ('01 October 2020', '01 October 2023'),
 ('01 July 2010', '30 October 2019'),
 ('05 May 2017', '09 October 2020'),
 ('01 September 2022', '15 June 2023'),
 ('01 February 2024', '01 March 2026'),
 ('11 May 2016', '13 December 2017'),
 ('2016-02', '2017-05'),
 ('13 July 2017', '30 June 2026'),
 ('2004-01', '2005-01'),
 ('09 February 2023', '17 November 2024'),
 ('2015-03', '2016-12'),
 ('13 June 2013', '02 November 2021'),
 ('13 June 2018', '24 December 2018'),
 ('2009-01', '2009-12'),
 ('2009-08', '2011-07'),
 ('2009-09', '2010-03'),
 ('02 December 2021', '31 December 2023'),
 ('2003-03', '2004-04'),
 ('30 May 2023', '30 May 2025'),
 ('15 July 2019', '01 September 2020'),
 ('15 April 2020', '31 May 2020'),
 ('2014-09', '2019-02'),
 ('01 July 2023', '01 December 2024'),
 ('01 January 2018', '31 December 2024'),
 ('06 May 2022', '06 November 202

In [0]:
#Create date without the Header

#create Date Header

Date_Header = rdd3.first()

print(Date_Header)

('Start Date', 'Completion Date')


In [0]:
rdd4 = rdd3.map(lambda x: x[:2])

rdd4.take(3)

Out[105]: [('Start Date', 'Completion Date'),
 ('20 August 2018', '28 February 2019'),
 ('25 October 2023', '2024-08')]

In [0]:
#Function to clean the date and convert to a consistent date format

from datetime import datetime

# Define a function to clean date data
def clean_date(date_str):
    if not date_str:
        return ""
        
    formats = ['%d %B %Y', '%Y %m %d', '%d %m %Y', '%Y-%m']
    for fmt in formats:
        try:
            return datetime.strptime(date_str, fmt).strftime('%d %B %Y')
        except ValueError:
            pass
    return date_str

# Clean the date data in the RDD
cleaned_dates = rdd3.map(lambda x: (clean_date(x[0]), clean_date(x[1])))

# Collect and print the cleaned date data
for date_pair in cleaned_dates.collect():
    print(date_pair)

('Start Date', 'Completion Date')
('20 August 2018', '28 February 2019')
('25 October 2023', '01 August 2024')
('01 August 2013', '01 December 2014')
('01 October 2020', '01 October 2023')
('01 July 2010', '30 October 2019')
('05 May 2017', '09 October 2020')
('01 September 2022', '15 June 2023')
('01 February 2024', '01 March 2026')
('11 May 2016', '13 December 2017')
('01 February 2016', '01 May 2017')
('13 July 2017', '30 June 2026')
('01 January 2004', '01 January 2005')
('09 February 2023', '17 November 2024')
('01 March 2015', '01 December 2016')
('13 June 2013', '02 November 2021')
('13 June 2018', '24 December 2018')
('01 January 2009', '01 December 2009')
('01 August 2009', '01 July 2011')
('01 September 2009', '01 March 2010')
('02 December 2021', '31 December 2023')
('01 March 2003', '01 April 2004')
('30 May 2023', '30 May 2025')
('15 July 2019', '01 September 2020')
('15 April 2020', '31 May 2020')
('01 September 2014', '01 February 2019')
('01 July 2023', '01 December 202

In [0]:
## Create an rdd excluding the 'Start" and 'Completion" date

rdd5 = rdd2.map(lambda x: x[:12])

rdd5.take(1)

Out[107]: [['NCT Number ',
  ' Study Title ',
  ' Acronym ',
  ' Study Status ',
  ' Conditions ',
  ' Interventions ',
  ' Sponsor ',
  ' Collaborators ',
  ' Enrollment ',
  ' Funder Type ',
  ' Study Type ',
  ' Study Design']]

In [0]:
#Zip RDDs together and flatten the rdd

# Zip the RDDs together column-wise
zipped_rdd = rdd5.zip(cleaned_dates)

# Add start and completion date columns to each tuple in the zipped RDD as a tuple
flattened_rdd = zipped_rdd.map(lambda x: x[0] + list(x[1]))

# Output the result
flattened_rdd.take(20)

Out[108]: [['NCT Number ',
  ' Study Title ',
  ' Acronym ',
  ' Study Status ',
  ' Conditions ',
  ' Interventions ',
  ' Sponsor ',
  ' Collaborators ',
  ' Enrollment ',
  ' Funder Type ',
  ' Study Type ',
  ' Study Design',
  'Start Date',
  'Completion Date'],
 ['"NCT03630471 ',
  ' Effectiveness of a Problem-solving Intervention for Common Adolescent Mental Health Problems in India ',
  ' PRIDE ',
  ' COMPLETED ',
  ' Mental Health Issue (E.G., Depression, Psychosis, Personality Disorder, Substance Abuse) ',
  " BEHAVIORAL: PRIDE 'Step 1' problem-solving intervention|BEHAVIORAL: Enhanced usual care ",
  ' Sangath ',
  ' Harvard Medical School (HMS and HSDM)|London School of Hygiene and Tropical Medicine ',
  ' 250 ',
  ' OTHER ',
  ' INTERVENTIONAL ',
  ' Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (INVESTIGATOR',
  '20 August 2018',
  '28 February 2019'],
 ['"NCT05992571 ',
  ' Oral Ketone Monoester Supplementation and Resting-state Brain Connectivity '

Exploratory data analysis

In [0]:
#Check the count of values by column header

# Extract the header row
header_row = flattened_rdd.first()

# Count the total number of non-empty values for each column
column_counts = flattened_rdd.flatMap(lambda row: zip(header_row, row)) \
                   .filter(lambda x: x[1] != '') \
                   .map(lambda x: (x[0], 1)) \
                   .reduceByKey(lambda a, b: a + b) \
                   .collect()

# Output the column counts
for column, count in column_counts:
    print(f"Column '{column}': {count}")

Column 'NCT Number ': 478698
Column ' Interventions ': 477801
Column ' Sponsor ': 477801
Column ' Collaborators ': 477775
Column ' Study Design': 467588
Column ' Study Title ': 478698
Column ' Study Status ': 478698
Column ' Funder Type ': 475705
Column ' Study Type ': 472627
Column 'Completion Date': 438184
Column ' Acronym ': 478697
Column ' Conditions ': 477815
Column ' Enrollment ': 477049
Column 'Start Date': 471145


In [0]:
#Likewise, Check missing values by column headers

# Extract the header row
header_row = flattened_rdd.first()

# Count the total number of empty values for each column
empty_counts = flattened_rdd.flatMap(lambda row: zip(header_row, row)) \
                  .filter(lambda x: x[1] == '') \
                  .map(lambda x: (x[0], 1)) \
                  .reduceByKey(lambda a, b: a + b) \
                  .collect()

# Output the column counts
for column, count in empty_counts:
    print(f"Column '{column}': {count}")

Column ' Study Design': 10187
Column ' Interventions ': 882
Column ' Sponsor ': 882
Column ' Collaborators ': 908
Column ' Study Type ': 5174
Column 'Completion Date': 12204
Column ' Funder Type ': 2096
Column 'Start Date': 5182
Column ' Enrollment ': 752
Column ' Conditions ': 883
Column ' Acronym ': 1


In [0]:
#Question 1: What is the Number of Distinct Studies in the Dataset

# Extract the header row
header_row = flattened_rdd.first()

# Find the index of the 'Study Title' column
study_title_index = header_row.index(' Study Title ')

# Count the number of distinct studies
distinct_study_count = flattened_rdd.map(lambda row: row[study_title_index]) \
                          .distinct() \
                          .count()

# Output the number of distinct studies
print("Number of distinct studies:", distinct_study_count)

Number of distinct studies: 476396


In [0]:
#Question 2: Frequency of Types of studies in dataset

# Extract the header row
header_row = rdd2.first()

# Calculate the number of fields in the header row
num_fields_header = len(header_row)

# Filter out rows with incomplete fields
complete_rows_rdd = rdd2.filter(lambda row: len(row) == num_fields_header)

# Extract the index of the 'Study Type' column
study_type_index = header_row.index(' Study Type ')

# Map each row to extract the study type
study_types_rdd = complete_rows_rdd.map(lambda row: row[study_type_index])
non_empty_study_types_rdd = study_types_rdd.filter(lambda study_type: study_type != '')
study_type_counts_rdd = non_empty_study_types_rdd.map(lambda study_type: (study_type, 1))
study_type_frequencies_rdd = study_type_counts_rdd.reduceByKey(lambda x, y: x + y)
sorted_study_type_frequencies_rdd = study_type_frequencies_rdd.map(lambda x: (x[1], x[0])) \
                                                            .sortByKey(ascending=False) \
                                                            .map(lambda x: (x[1], x[0]))

for study_type, frequency in sorted_study_type_frequencies_rdd.collect():
    print(f"Study Type: {study_type}, Frequency: {frequency}")

Study Type:  INTERVENTIONAL , Frequency: 29569
Study Type:  Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (PARTICIPANT, Frequency: 9646
Study Type:  OBSERVATIONAL , Frequency: 7024
Study Type:  CARE_PROVIDER, Frequency: 6760
Study Type:  INVESTIGATOR, Frequency: 5927
Study Type:  Allocation: RANDOMIZED|Intervention Model: CROSSOVER|Masking: DOUBLE (PARTICIPANT, Frequency: 1687
Study Type:  Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (INVESTIGATOR, Frequency: 1468
Study Type:  Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (CARE_PROVIDER, Frequency: 341
Study Type:  Allocation: RANDOMIZED|Intervention Model: SINGLE_GROUP|Masking: DOUBLE (PARTICIPANT, Frequency: 302
Study Type:  Allocation: RANDOMIZED|Intervention Model: FACTORIAL|Masking: DOUBLE (PARTICIPANT, Frequency: 223
Study Type:  Allocation: RANDOMIZED|Intervention Model: SEQUENTIAL|Masking: DOUBLE (PARTICIPANT, Frequency: 193
Study Type:  Allocation: RANDOMIZED|I

In [0]:
#Question 3: Top 5 conditions by frequencies

# Extract the header row
header_row = rdd2.first()

# Calculate the number of fields in the header row
num_fields_header = len(header_row)

# Filter out rows with incomplete fields
complete_rows_rdd = rdd2.filter(lambda row: len(row) == num_fields_header)

# Extract the index of the 'Conditions' column
conditions_index = header_row.index(' Conditions ')

# Map each row to extract the conditions
conditions_rdd = complete_rows_rdd.map(lambda row: row[conditions_index])
non_empty_conditions_rdd = conditions_rdd.filter(lambda condition: condition != '')
conditions_counts_rdd = non_empty_conditions_rdd.map(lambda condition: (condition, 1))
conditions_frequencies_rdd = conditions_counts_rdd.reduceByKey(lambda x, y: x + y)
sorted_conditions_frequencies_rdd = conditions_frequencies_rdd.map(lambda x: (x[1], x[0])) \
                                                            .sortByKey(ascending=False) \
                                                            .map(lambda x: (x[1], x[0]))

#Top 5
top_5_conditions = sorted_conditions_frequencies_rdd.collect()

for condition, frequency in top_5_conditions:
    print(f"Condition: {condition}, Frequency: {frequency}")


Condition:  Breast Cancer , Frequency: 409
Condition:  Obesity , Frequency: 261
Condition:  Prostate Cancer , Frequency: 257
Condition:  Healthy , Frequency: 244
Condition:  Coronary Artery Disease , Frequency: 229
Condition:  Heart Failure , Frequency: 220
Condition:  Stroke , Frequency: 205
Condition:  Atrial Fibrillation , Frequency: 187
Condition:  Asthma , Frequency: 185
Condition:  Hypertension , Frequency: 161
Condition:  HIV Infections , Frequency: 161
Condition:  Type 2 Diabetes , Frequency: 153
Condition:  HIV , Frequency: 149
Condition:  COVID-19 , Frequency: 149
Condition:  Cancer , Frequency: 144
Condition:  Diabetes Mellitus, Type 2 , Frequency: 143
Condition:  Parkinson Disease , Frequency: 137
Condition:  Multiple Sclerosis , Frequency: 135
Condition:  Rheumatoid Arthritis , Frequency: 131
Condition:  Multiple Myeloma , Frequency: 125
Condition:  Depression , Frequency: 121
Condition:  Colorectal Cancer , Frequency: 118
Condition:  Covid19 , Frequency: 106
Condition:  T

In [0]:
# Extract the header row
header_row = rdd2.first()

# Find the index position of 'Conditions' column
conditions_index = None
for i, col in enumerate(header_row):
    if 'Conditions' in col:
        conditions_index = i
        break

# Filter out rows with incomplete fields
complete_rows_rdd = rdd2.filter(lambda row: len(row) == len(header_row))

# Map each row to extract the conditions
if conditions_index is not None:
    conditions_rdd = complete_rows_rdd.map(lambda row: row[conditions_index])
    #non_empty_conditions_rdd = conditions_rdd.filter(lambda condition: condition != '')
    conditions_counts_rdd = conditions_rdd.map(lambda condition: (condition, 1))
    conditions_frequencies_rdd = conditions_counts_rdd.reduceByKey(lambda x, y: x + y)
    sorted_conditions_frequencies_rdd = conditions_frequencies_rdd.map(lambda x: (x[1], x[0])) \
                                                                .sortByKey(ascending=False) \
                                                                .map(lambda x: (x[1], x[0]))

    # Top 5
    top_5_conditions = sorted_conditions_frequencies_rdd.collect()

    for condition, frequency in top_5_conditions:
        print(f"Condition: {condition}, Frequency: {frequency}")
else:
    print("Conditions column not found in the header row.")


Condition:  Breast Cancer , Frequency: 409
Condition:  Obesity , Frequency: 261
Condition:  Prostate Cancer , Frequency: 257
Condition:  Healthy , Frequency: 244
Condition:  Coronary Artery Disease , Frequency: 229
Condition:  Heart Failure , Frequency: 220
Condition:  Stroke , Frequency: 205
Condition:  Atrial Fibrillation , Frequency: 187
Condition:  Asthma , Frequency: 185
Condition:  Hypertension , Frequency: 161
Condition:  HIV Infections , Frequency: 161
Condition:  Type 2 Diabetes , Frequency: 153
Condition:  HIV , Frequency: 149
Condition:  COVID-19 , Frequency: 149
Condition:  Cancer , Frequency: 144
Condition:  Diabetes Mellitus, Type 2 , Frequency: 143
Condition:  Parkinson Disease , Frequency: 137
Condition:  Multiple Sclerosis , Frequency: 135
Condition:  Rheumatoid Arthritis , Frequency: 131
Condition:  Multiple Myeloma , Frequency: 125
Condition:  Depression , Frequency: 121
Condition:  Colorectal Cancer , Frequency: 118
Condition:  Covid19 , Frequency: 106
Condition:  T